In [12]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
from scipy.optimize import curve_fit

In [2]:
exps = ['GramA 2.6nM','GramA 260pM20210920','GUV','OmpF']
path_identifier = ['3nM','260pM20210920','GUV','OmpF']

paths = np.array(glob('Permeabilities*.csv'))
print(paths)
exp_hash = {}
positions = np.arange(len(paths))

for i,path_id in enumerate(path_identifier):
    for position,path in enumerate(paths[positions]):
        if path.find(path_id) != -1:
            
            exp_hash[exps[i]] = path
#             positions = np.concatenate((positions[:position],positions[position+1:]))
            print(positions)

['Permeabilities_GramicidinA_260pM20210920.csv'
 'Permeabilities_GramicidinA_3nM_20210920.csv'
 'Permeabilities_OmpF_20210427.csv' 'Permeabilities_GUV.csv']
[0 1 2 3]
[0 1 2 3]
[0 1 2 3]
[0 1 2 3]


In [13]:
exp_hash

{'GramA 2.6nM': 'Permeabilities_GramicidinA_3nM_20210920.csv',
 'GramA 260pM20210920': 'Permeabilities_GramicidinA_260pM20210920.csv',
 'GUV': 'Permeabilities_GUV.csv',
 'OmpF': 'Permeabilities_OmpF_20210427.csv'}

In [19]:

dfs = {exps[i]: pd.read_csv(exp_hash[exps[i]]) for i in range(len(exps))}

In [5]:
%matplotlib qt
fig, axs  = plt.subplots(1,len(dfs.keys()))

for i,key in enumerate(dfs.keys()):
    
    rel_err = dfs[key]['dP/P'].to_numpy().astype(float)
    P = dfs[key]['P cms^-1'].to_numpy().astype(float)
    rel_err = rel_err[P > 0]
    P = P[P > 0]

    
    axs[i].hist(np.log10(P[rel_err < 0.1]),'sturges')
    
    x_lab = [-8,-6.5,-5]
    y_lab = [0,5,10]
    
    axs[i].set_xticks(x_lab)
    axs[i].set_xticklabels(x_lab,fontsize = 15)
    
    axs[i].set_yticks(y_lab)
    axs[i].set_yticklabels(y_lab,fontsize = 15)
    
    axs[i].set_xlim([-8.5,-4.5])
    axs[i].set_ylim([0,11])
    
    axs[i].set_title(key)

In [83]:
print(plt.rcParams['axes.prop_cycle'].by_key()['color'])

['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']


In [32]:
# now when happy with the settings arrange on one plot
fig, ax  = plt.subplots(1,1)

bins = {'GramA 2.6nM': 8, 'GramA 260pM20210920': 5,'GUV':8,'OmpF':6}
for i,key in enumerate(dfs.keys()):
    if key not in ['GUV','OmpF']:
        continue
    rel_err = dfs[key]['dP/P'].to_numpy().astype(float)
    P = dfs[key]['P cms^-1'].to_numpy().astype(float)
    rel_err = rel_err[P > 0]
    P = P[P > 0]

    color = plt.rcParams['axes.prop_cycle'].by_key()['color'][i]
    if i == 2:
        color = 'g'
    if i == 3:
        color = 'C1'
    ax.hist(np.log10(P[rel_err < 0.1]),bins[key],label = key,alpha = 0.3,color = color,
            edgecolor = color)
    
x_lab = [-8,-6.5,-5]
y_lab = [0,10,20]

ax.set_xticks(x_lab)
ax.set_xticklabels(x_lab,fontsize = 15)

ax.set_yticks(y_lab)
ax.set_yticklabels(y_lab,fontsize = 15)

ax.set_xlim([-8.5,-4.5])
ax.set_ylim([0,20])

# ax.set_title(key)

ax.set_xlabel('$Log_{10} (P [cms^{-1}])$',fontsize = 18)
ax.set_ylabel('# GUVs',fontsize = 18)
# plt.legend()

plt.tick_params(direction = 'in',top= True, right = True,left = True, bottom = True, length = 6)


In [26]:
# now when happy with the settings arrange on one plot
fig, ax  = plt.subplots(1,1)

bins = {'GramA 2.6nM': 9, 'GramA 260pM20210920': 5,'GUV':8,'OmpF':6}
for i,key in enumerate(dfs.keys()):
    print(key)
    if key not in ['GramA 2.6nM','GramA 260pM20210920']:
        continue
    rel_err = dfs[key]['dP/P'].to_numpy().astype(float)
    P = dfs[key]['P cms^-1'].to_numpy().astype(float)
    rel_err = rel_err[P > 0]
    P = P[P > 0]

    color = plt.rcParams['axes.prop_cycle'].by_key()['color'][i]
    
    ax.hist(np.log10(P[rel_err < 0.1]),bins[key],label = key,alpha = 0.3,
            edgecolor = color)
    
x_lab = [-7,-6.5,-6]
y_lab = [0,10,20,30]

ax.set_xticks(x_lab)
ax.set_xticklabels(x_lab,fontsize = 15)

ax.set_yticks(y_lab)
ax.set_yticklabels(y_lab,fontsize = 15)

ax.set_xlim([-7.5,-5.5])
ax.set_ylim([0,31])

# ax.set_title(key)

ax.set_xlabel('$Log_{10} (P [cms^{-1}])$',fontsize = 18)
ax.set_ylabel('# GUVs',fontsize = 18)
# plt.legend(

plt.tick_params(direction = 'in',top= True, right = True,left = True, bottom = True, length = 6)


GramA 2.6nM
GramA 260pM20210920
GUV
OmpF


# Fit conductance vs concentration for OmpF


In [25]:
dfGvsK = pd.DataFrame({'[K][molm^-3]': [100,500,1000,1500],'G [S]':[0.5e-9,2e-9,4.5e-9,5.25e-9]})

In [26]:
# fitting function

def GfromC(C,P):
    F = 96485
    R = 8.31
    T = 298
    
    return (F**2*C*P)/(R*T)

guess = [2e-18]

# fit using curve fitabs
params = curve_fit(GfromC,dfGvsK['[K][molm^-3]'],dfGvsK['G [S]'],p0 = guess)

(array([1.01743076e-18]), array([[4.80305164e-39]]))

In [27]:
#{ key: [Conductance,Potassium concentration when measured]}

conductances = {'OmpF':[8e-10,100],'GramA':[1.4e-11,100]}


def PfromG(G,C,R = 8.31,T = 298,F = 96485):
    #make sure C is given as molm^-3 for units to make sense
    #also G given as Siemens (Amperes/Volts)
    
    #returns P in m^3/s
    return G*(R*T)/(F**2*C)
   
    
    
exp_one_P = {key: PfromG(*conductances[key]) for key in conductances.keys()}

In [28]:
# use fit for permeability of one OmpF pore

exp_one_P['OmpF'] = params[0]

In [29]:
App = 3.16 #um^2
for key in exps:
    
    if key == 'GUV':
        continue
        
    if key.find('GramA') != -1:
        id_ = 'GramA'
        
    else:
        id_  = key
        
    dfs[key]['P1_exp ms^-1'] = exp_one_P[id_]/(dfs[key]['A']*App*1e-12)
    dfs[key]['N'] = dfs[key]['P ms^-1']/dfs[key]['P1_exp ms^-1']

In [30]:
# process Number of pores per GUV to make sense
key = 'OmpF'


dfs[key]['N'].iloc[dfs[key]['N'].to_numpy().astype(float) < 0] = 0

/home/marcus/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [35]:
%matplotlib qt
fig, axs  = plt.subplots(1,len(dfs.keys()))

for i,key in enumerate(dfs.keys()):
    
    if key == 'GUV':
        continue
    rel_err = dfs[key]['dP/P'].to_numpy().astype(float)
    N = dfs[key]['N'].to_numpy().astype(float)
    rel_err = rel_err[N > 0]
    N = N[N > 0]

    if key == 'OmpF':
        axs[i].hist(N[rel_err < 0.1],'sqrt')
        
    else:
        axs[i].hist(np.log10(N[rel_err < 0.1]),'sqrt')
    axs[i].set_title(key)

In [37]:
# Gramicidin A plot

keys = ['GramA 2.6nM','GramA 260pM20210920']

fig, ax = plt.subplots(1,1)


for i,key in enumerate(keys):

    rel_err = dfs[key]['dP/P'].to_numpy().astype(float)
    N = dfs[key]['N'].to_numpy().astype(float)
    
    color = plt.rcParams['axes.prop_cycle'].by_key()['color'][i]
    ax.hist(np.log10(N[rel_err < 0.1]),5,label = key,alpha = 0.3,
            edgecolor = color)
    

x_ticks = [2,3,4]
y_ticks= [0,10,20,30]

  
plt.tick_params(direction = 'in',top= True, right = True,left = True, bottom = True, length = 6)

ax.set_xticks(x_ticks)
ax.set_xticklabels(x_ticks,fontsize = 15)
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_ticks,fontsize = 15)

ax.set_ylabel('# GUVs',fontsize = 18)
ax.set_xlabel('$Log_{10}$(No. of pores per GUV)', fontsize = 18)

ax.set_xlim([1.9,4.1])
# plt.legend()


(1.9, 4.1)

In [43]:
# OmpF plot
keys = ['OmpF']

fig, ax = plt.subplots(1,1)


for i,key in enumerate(keys):

    rel_err = dfs[key]['dP/P'].to_numpy().astype(float)
    N = dfs[key]['N'].to_numpy().astype(float)
    
    color = plt.rcParams['axes.prop_cycle'].by_key()['color'][i]
    ax.hist(N[rel_err < 0.1],6,label = key,alpha = 0.3,
            edgecolor = color)
    

x_ticks = [0,8,16,24]
y_ticks= [0,4,8]

  
plt.tick_params(direction = 'in',top= True, right = True,left = True, bottom = True, length = 6)

ax.set_xticks(x_ticks)
ax.set_xticklabels(x_ticks,fontsize = 15)
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_ticks,fontsize = 15)

ax.set_ylabel('# GUVs',fontsize = 18)
ax.set_xlabel('$No. of pores per GUV', fontsize = 18)

# ax.set_xlim([1.9,4.1])
# plt.legend()
# 

Text(0.5, 0, '$No. of pores per GUV')